In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [3]:
import sys
sys.path.append('../')

from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

## Krippendorff's alpha

In [57]:
#read in seattle validation labels
seattle_labels = gpd.read_file('data/round-3/seattle-validation-labels-round3-230927.geojson')
seattle_labels = seattle_labels.drop(columns=['time_created'])
seattle_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,8263,519,Greenwood,Obstacle,47.697910,-122.346878,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34688 47.69791)
1,Devon Snyder,8264,519,Greenwood,Obstacle,47.697910,-122.346840,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34684 47.69791)
2,Devon Snyder,8265,519,Greenwood,SurfaceProblem,47.697731,-122.347191,u54hhkI1_JMvO6PSVuls_Q,None,1,f,cracks,None,POINT (-122.34719 47.69773)
3,Devon Snyder,8266,519,Greenwood,Obstacle,47.697914,-122.346817,-bwvo2vUzCb2YOsT0bKDGQ,None,5,t,sign,None,POINT (-122.34682 47.69791)
4,Devon Snyder,8267,519,Greenwood,SurfaceProblem,47.697880,-122.346687,-bwvo2vUzCb2YOsT0bKDGQ,None,1,f,utility panel,None,POINT (-122.34669 47.69788)


In [23]:
#count the cluster numbers for each label type
def count_clusters(labels, label_type_id):
    df = cluster_label_type_at_index(labels,label_type_id)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    return df1.cluster_id.nunique()

In [24]:
cluster_count_seattle = {}
for i in [2,4,5]:
    clusters =  count_clusters(seattle_labels, i)
    cluster_count_seattle[label_types[i]] = clusters

In [26]:
cluster_count_seattle

{'Problem': 291, 'SurfaceProblem': 259, 'Obstacle': 16}

In [9]:
#count the numbers for eanch label type
seattle_labels.label_type.value_counts()

SurfaceProblem    728
Obstacle           80
Name: label_type, dtype: int64

In [10]:
#read in seattle streets
seattle_streets = gpd.read_file("data/round-3/seattle-streets-230927.geojson")

In [11]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=20):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=20):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [12]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to seattle_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [13]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [15]:
#calculate krippendorff's alpha
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [27]:
alpha_seattle = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(seattle_labels,i, seattle_streets, 2285)
    alpha_seattle[label_types[i]] = alpha

In [29]:
#30 ft segments
alpha_seattle

{'Problem': 0.661, 'SurfaceProblem': 0.66, 'Obstacle': 0.389}

### surface problem and obstacle for severity level > = 4

In [44]:
#take out label type = surfaceproblem and obstacle
seattle_problems= seattle_labels[(seattle_labels['label_type'] == 'SurfaceProblem') | (seattle_labels['label_type'] == 'Obstacle') ]
#select only severity >=4
seattle_problems_4 = seattle_problems[seattle_problems['severity'] >= 4]
seattle_problems_4

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,8263,519,Greenwood,Obstacle,47.697910,-122.346878,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34688 47.69791)
1,Devon Snyder,8264,519,Greenwood,Obstacle,47.697910,-122.346840,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34684 47.69791)
3,Devon Snyder,8266,519,Greenwood,Obstacle,47.697914,-122.346817,-bwvo2vUzCb2YOsT0bKDGQ,None,5,t,sign,None,POINT (-122.34682 47.69791)
79,Devon Snyder,8345,6,Ballard,Obstacle,47.677937,-122.402039,CTpFmcG_EoiuZibDGbsm1g,None,4,f,parked car,None,POINT (-122.40204 47.67794)
108,Devon Snyder,8374,757,Windermere,Obstacle,47.665848,-122.270592,ejvoKyXM3OaM22knR2uE0w,None,4,f,parked car,None,POINT (-122.27059 47.66585)
123,Devon Snyder,8390,302,Queen Anne,SurfaceProblem,47.645855,-122.358192,bSDzC_ivz3I9fW_RpO-TLA,None,5,f,grass,debris all over the sidewalk,POINT (-122.35819 47.64585)
137,Devon Snyder,8404,275,Queen Anne,SurfaceProblem,47.646854,-122.356972,en_aPYvKeegnvUtw47bNbg,None,4,f,"very broken,grass,cracks",None,POINT (-122.35697 47.64685)
139,Devon Snyder,8406,301,Queen Anne,SurfaceProblem,47.645679,-122.358253,jNpQIPe1koWXyGz4g4aHUg,None,4,t,None,debris all over the sidewalk,POINT (-122.35825 47.64568)
140,Devon Snyder,8407,301,Queen Anne,SurfaceProblem,47.645531,-122.358276,xoHROUHOQaf8FiphiwLi0g,None,5,t,None,debris all over the sidewalk,POINT (-122.35828 47.64553)
141,Devon Snyder,8408,301,Queen Anne,SurfaceProblem,47.645477,-122.358261,yTIxSIfO8My54hj_Pp49jA,None,5,t,None,debris all over the sidewalk,POINT (-122.35826 47.64548)


In [39]:
alpha_seattle_problems_4 = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(seattle_problems_4,i, seattle_streets, 2285)
    alpha_seattle_problems_4[label_types[i]] = alpha

In [41]:
alpha_seattle_problems_4

{'Problem': 0.43, 'SurfaceProblem': 0.294, 'Obstacle': 0.663}

### surface problem and obstacle for severity level != 1

In [53]:
#take out severity ==1
seattle_problems_1 = seattle_problems[seattle_problems['severity'] != 1]
seattle_problems_1

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,8263,519,Greenwood,Obstacle,47.697910,-122.346878,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34688 47.69791)
1,Devon Snyder,8264,519,Greenwood,Obstacle,47.697910,-122.346840,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34684 47.69791)
3,Devon Snyder,8266,519,Greenwood,Obstacle,47.697914,-122.346817,-bwvo2vUzCb2YOsT0bKDGQ,None,5,t,sign,None,POINT (-122.34682 47.69791)
9,Devon Snyder,8273,519,Greenwood,SurfaceProblem,47.697868,-122.346077,rbyOYvxCT3V-yu36bl6H-g,None,2,f,cracks,None,POINT (-122.34608 47.69787)
12,Devon Snyder,8276,519,Greenwood,SurfaceProblem,47.697758,-122.345909,8_QhZrK8HjNQKU89gRliiw,None,2,f,cracks,None,POINT (-122.34591 47.69776)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,mikey,9257,785,Windermere,Obstacle,47.664925,-122.270256,ZkFu2C_hMTHhDN9zo9J1Xw,None,2,f,vegetation,None,POINT (-122.27026 47.66492)
792,mikey,9263,757,Windermere,SurfaceProblem,47.665413,-122.270012,NRu8hHxXG5_brpYK3HoiXw,None,2,f,cracks,None,POINT (-122.27001 47.66541)
795,mikey,9266,757,Windermere,Obstacle,47.665867,-122.270599,oPuBSTkgTXQJTY6vgXPa7Q,None,5,f,parked car,None,POINT (-122.27060 47.66587)
797,mikey,9268,757,Windermere,SurfaceProblem,47.665905,-122.270683,ejvoKyXM3OaM22knR2uE0w,None,3,f,height difference,None,POINT (-122.27068 47.66590)


In [54]:
alpha_seattle_problems_1 = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(seattle_problems_1,i, seattle_streets, 2285)
    alpha_seattle_problems_1[label_types[i]] = alpha

In [56]:
alpha_seattle_problems_1

{'Problem': 0.572, 'SurfaceProblem': 0.559, 'Obstacle': 0.394}

## HTML

In [50]:
for i in [2,4,5]:
    labels= map_clusters(seattle_problems,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/seattle_"+label_types[i]+"_round-3_230927.html")